<a href="https://colab.research.google.com/github/omkarpat/BotBuilder-Location/blob/master/Baseline_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Colab settings/mount
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My\ Drive/CSE\ 240/Project/Data/spectrogram_images/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/CSE 240/Project/Data/spectrogram_images


In [0]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from keras import optimizers
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, SeparableConv2D, MaxPooling3D, Conv3D, DepthwiseConv2D

In [0]:
base_path = "."
concentration_data_array = None
relaxed_data_array = None
#relaxed_data_array = np.load("relaxed_array_updated_spectrograms_final.npy")
for filename in os.listdir(base_path):
  if "concentration_array_updated_spectrograms" in filename and "final" not in filename:
    print(filename)
    concentration_data_array = np.concatenate((concentration_data_array, np.load(filename))) if concentration_data_array is not None else np.load(filename)
  if "relaxed_array_updated_spectrograms" in filename:
    print(filename)
    relaxed_data_array = np.concatenate((relaxed_data_array, np.load(filename))) if relaxed_data_array is not None else np.load(filename)

concentration_array_updated_spectrograms_1.npy
concentration_array_updated_spectrograms_2.npy
concentration_array_updated_spectrograms_3.npy
concentration_array_updated_spectrograms_4.npy
concentration_array_updated_spectrograms_5.npy
concentration_array_updated_spectrograms_6.npy
relaxed_array_updated_spectrograms_1.npy
relaxed_array_updated_spectrograms_2.npy
relaxed_array_updated_spectrograms_3.npy
relaxed_array_updated_spectrograms_4.npy
relaxed_array_updated_spectrograms_5.npy
relaxed_array_updated_spectrograms_final.npy


In [0]:
print(concentration_data_array.shape, relaxed_data_array.shape)

(2160, 29, 43, 42) (2124, 29, 43, 42)


In [0]:
X = np.concatenate((relaxed_data_array, concentration_data_array))
relaxed_data_array, concentration_data_array = None, None

In [0]:
Y = np.concatenate((np.zeros((2160,1)),np.ones((2124,1))))
print(X.shape, Y.shape)

(4284, 29, 43, 42) (4284, 1)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=42)
X, Y = None, None
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.15, random_state=42)
print(X_train.shape, X_test.shape, X_valid.shape, y_train.shape, y_test.shape, y_valid.shape)

(3094, 29, 43, 42) (643, 29, 43, 42) (547, 29, 43, 42) (3094, 1) (643, 1) (547, 1)


In [0]:
model2 = Sequential()
model2.add(Conv2D(64, kernel_size=5, strides=(1,1), padding='valid', activation='relu', input_shape=(29, 43, 42)))
model2.add(Conv2D(128, kernel_size=3, activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Flatten())
model2.add(Dense(50, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
sgd = optimizers.SGD(lr=0.0000001, momentum=0.7)
model2.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
model2.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=50, batch_size=32, shuffle=True)

Train on 3094 samples, validate on 547 samples
Epoch 1/50
3094/3094 [==============================] - 62s 20ms/step - loss: 7.1788 - acc: 0.5090 - val_loss: 6.1989 - val_acc: 0.5411
Epoch 2/50
3094/3094 [==============================] - 61s 20ms/step - loss: 6.1622 - acc: 0.5133 - val_loss: 5.3568 - val_acc: 0.5521
Epoch 3/50
3094/3094 [==============================] - 62s 20ms/step - loss: 6.0394 - acc: 0.5239 - val_loss: 5.1607 - val_acc: 0.5503
Epoch 4/50
3094/3094 [==============================] - 63s 20ms/step - loss: 5.7652 - acc: 0.5155 - val_loss: 5.3374 - val_acc: 0.5503
Epoch 5/50
3094/3094 [==============================] - 62s 20ms/step - loss: 5.7171 - acc: 0.5152 - val_loss: 5.2823 - val_acc: 0.5393
Epoch 6/50
3094/3094 [==============================] - 62s 20ms/step - loss: 5.3091 - acc: 0.5265 - val_loss: 6.1897 - val_acc: 0.5265
Epoch 7/50
3094/3094 [==============================] - 62s 20ms/step - loss: 5.2183 - acc: 0.5265 - val_loss: 4.9344 - val_acc: 0.5448
E

In [44]:
model2.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=25, batch_size=64, shuffle=True)

Train on 3094 samples, validate on 547 samples
Epoch 1/25
3094/3094 [==============================] - 55s 18ms/step - loss: 2.2543 - acc: 0.6341 - val_loss: 3.5949 - val_acc: 0.5375
Epoch 2/25
3094/3094 [==============================] - 57s 18ms/step - loss: 2.2722 - acc: 0.6399 - val_loss: 3.5704 - val_acc: 0.5430
Epoch 3/25
3094/3094 [==============================] - 57s 18ms/step - loss: 2.2573 - acc: 0.6412 - val_loss: 3.6393 - val_acc: 0.5356
Epoch 4/25
3094/3094 [==============================] - 57s 18ms/step - loss: 2.2804 - acc: 0.6416 - val_loss: 3.5848 - val_acc: 0.5375
Epoch 5/25
3094/3094 [==============================] - 58s 19ms/step - loss: 2.2530 - acc: 0.6435 - val_loss: 3.5613 - val_acc: 0.5411
Epoch 6/25
3094/3094 [==============================] - 57s 18ms/step - loss: 2.2756 - acc: 0.6435 - val_loss: 3.5689 - val_acc: 0.5393
Epoch 7/25
3094/3094 [==============================] - 56s 18ms/step - loss: 2.2742 - acc: 0.6390 - val_loss: 3.5629 - val_acc: 0.5430
E

In [45]:
metrics = model2.evaluate(X_test,y_test)
print(metrics)

643/643 [==============================] - 4s 6ms/step
[3.35662171462461, 0.5365474339962747]
